# Multi-Part Experiment 1 vs N

To use this notebook, you already should have run the following notebooks:
1. E1_0_Record_and_Segment.ipynb
2. E1_1_Multi_Part_Selection.ipynb

Summary:
1. This notebook uses already computed scores stored as npz files
2. For each part, compute scores at the front live (current_rgb vs first frame of the part)
3. Compute full trajectory from current state to the goal state

We compute rewards for the shapes: {'trapeze'} using scoring functions {'sum', 'prod'}

In [ ]:
import os
import copy
import time
import json
import shutil
import unittest
import subprocess
from pathlib import Path
import numpy as np

from scipy.spatial.transform import Rotation as R

from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.demo.demo_episode_recorder import record_sim
from flow_control.runner import evaluate_control
from flow_control.servoing.module import ServoingModule
from flow_control.servoing.playback_env_servo import PlaybackEnvServo
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout
import seaborn as sns
from tqdm import tqdm
import getpass


%matplotlib inline

experiment = "multi_part"

renderer = 'debug'
task_variant = 'rP'

def get_data_dir():
    username = getpass.getuser()
    if username == "argusm":
        return "/tmp/flow_experiments3"
    elif username == "nayakab":
        return "../tmp"

data_dir = get_data_dir()

root_dir = os.path.join(data_dir, experiment)

## Load all Demonstrations

In [ ]:
def get_recordings(directory):
    return sorted([os.path.join(directory, rec) for rec in os.listdir(directory) if os.path.isdir(os.path.join(directory, rec))])

recordings = get_recordings(root_dir)

In [ ]:
# Load the demonstration episodes
playbacks = [PlaybackEnvServo(rec) for rec in recordings[:]]

# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])

def update(demo_index, frame_index):
    image = playbacks[demo_index][frame_index].cam.get_image()[0]
    image_h.set_data(image)
    fig.canvas.draw_idle()
    print("wp_name:", playbacks[demo_index][frame_index].get_info()["wp_name"])
    fg_mask = playbacks[demo_index].get_fg_mask()
    if fg_mask is not None:
        print("percent fg:", np.mean(fg_mask)*100)
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

# Read scores (errors) from the stored files

In [ ]:
error_matrix_2_part = np.load(f"{root_dir}/error_matrix_2_part.npz")['arr_0']
score_matrix_2_part = 1 - error_matrix_2_part

error_matrix_3_part_1 = np.load(f"{root_dir}/error_matrix_3_part_1.npz")['arr_0']
score_matrix_3_part_1 = 1 - error_matrix_3_part_1

error_matrix_3_part_2 = np.load(f"{root_dir}/error_matrix_3_part_2.npz")['arr_0']
score_matrix_3_part_2 = 1 - error_matrix_3_part_2

In [ ]:
# Get good demonstrations and demonstration part information
def filter_demo(pb):
    return pb[-1].data['rew'] > 0 and np.mean(pb.get_fg_mask()) > 0.005

demo_good = [filter_demo(pb) for pb in playbacks]
good_demonstrations = np.where(demo_good)[0]

good_demonstrations = [int(x) for x in good_demonstrations]
live_seeds = good_demonstrations
print(good_demonstrations)

In [ ]:
# Load demo segmentation file
demo_seg_file = f'{root_dir}/demo_parts_manual2.json'
fp = open(demo_seg_file)
demo_2parts = json.load(fp)

demo_seg_file = f'{root_dir}/demo_parts_manual3.json'
fp = open(demo_seg_file)
demo_3parts = json.load(fp)

live_seeds = [int(key) for key in demo_2parts.keys()]
demo_keys = demo_2parts.keys()

## Compute live scores 
1. We compute live scores of the current state of the gripper wrt start frames of the part.
2. The part is identified by traj_idx in the function
3. demo_parts contains all the part information and the keyframes relating to each part

In [ ]:
from sklearn.preprocessing import minmax_scale

# Load Servoing Module
from flow_control.servoing.module import ServoingModule
control_config = dict(mode="pointcloud-abs-rotz", threshold=0.40)
servo_module = ServoingModule(recordings[0], control_config=control_config,
                              start_paused=False)

def compute_current_scores(playbacks, current_rgb, demo_parts, demo_good, traj_idx=0, live_seed=0):    
    sim_errors = np.ones(len(playbacks)) # lower is better
    mean_flows = np.zeros(len(playbacks))

    for demo_seed in good_demonstrations:
        start_idx = demo_parts[str(demo_seed)][traj_idx]['start']
        demo_rgb =  playbacks[demo_seed][start_idx].cam.get_image()[0]
        demo_mask =  playbacks[demo_seed].fg_masks[start_idx]
        error, mean_flow = similarity_from_reprojection(current_rgb, demo_rgb, demo_mask)
        sim_errors[demo_seed] = error
        mean_flows[demo_seed] = mean_flow
    errors_norm = normalize_errors(sim_errors, mean_flows, demo_good)
    scores = 1 - errors_norm
    
    return scores

def similarity_from_reprojection(live_rgb, demo_rgb, demo_mask, return_images=False):
    # evaluate the similarity via flow reprojection error
    flow = servo_module.flow_module.step(demo_rgb, live_rgb)
    warped = servo_module.flow_module.warp_image(live_rgb / 255.0, flow)
    diff = (warped - (demo_rgb / 255.0))
    error = np.linalg.norm((warped - (demo_rgb / 255.0)), axis=2) * demo_mask
    error = error.sum() / demo_mask.sum()
    mean_flow = np.linalg.norm(flow[demo_mask],axis=1).mean()
    if return_images:
        return error, mean_flow, flow, warped
    return error, mean_flow


def normalize_errors(errors, flows, demo_good):
    errors_l = errors[demo_good]
    mean_flows_l = flows[demo_good]
    errors_norm = np.ones(errors.shape)
    w = .5
    errors_norm[demo_good] = np.mean((1*minmax_scale(errors_l), w*minmax_scale(mean_flows_l)),axis=0)/(1+w)
    return errors_norm

In [ ]:
# Arrange keypoint information for all parts in a easy to use data structure
def split_keypoints(pb, demo_part, num_parts=2):
    demo_keep = sorted(list(pb.keep_dict.keys()))
    keep_all = copy.copy(pb.keep_dict)
    keep_parts = {}
    for p in demo_part:
        if p["start"] == 0:
            p_start = -1
        else:
            p_start = p["start"]

        parts = []
        for demo_index in demo_keep:
            if p_start < demo_index and p["end"] >= demo_index:
                parts.append(demo_index)   

        keep_parts[p["name"]] = parts
    
    if num_parts == 2:        
        # Need to also add the first frame of the 'insert' part to the 'locate' part
        keep_parts['locate'].append(keep_parts['insert'][0])
    elif num_parts == 3:
        # Need to also add the first frame of the 'insert' part to the 'locate' part
        keep_parts['grasp'].append(keep_parts['insert'][0])
        
    return keep_parts

keypoint_info_2parts = {}
keypoint_info_3parts = {}

for demo_seed in good_demonstrations:
    keypoint_info_2parts[demo_seed] = split_keypoints(playbacks[demo_seed], demo_2parts[str(demo_seed)])
    keypoint_info_3parts[demo_seed] = split_keypoints(playbacks[demo_seed], demo_3parts[str(demo_seed)], num_parts=3)

keypoint_info_3parts

## Search for best trajectory 

For this experiment, we only consider the 'sum' and 'prod' scoring functions.

The following function computes the cumulative scores starting at each part all the way to the goal state.

For example:

live_state ---- score_matrix_1 ---- score_matrix_2 ---- goal_state 

simplifies to 

live_scores ---- cscore[0] ---- cscore[1] ---- cscore[2]

1. cscore[2] just contains the goal scores
2. cscore[i] is an array of size n ** (num_parts - i)
3. score_matrices = [score_matrix_1, score_matrix_2, and so on]
4. Once the live scores are available, they can be used along with the corresponding cumulative score to find the best demonstration index

In [ ]:
# Compute cumulative scores staring from the goal state
# score_matrices should contain all the computed score matrices in order from live state to goal state

def get_cumulative_scores(scores_rear, score_matrices, score_fn='sum'):
    # Start from goal state, add goal scores to the list
    cumulative_scores = [scores_rear]    
    
    num_demos = scores_rear.shape[0]
    
    # Need to compute cumulative scores from the goal state. Need to reverse the order
    score_matrices.reverse()
    
    # Compute cumulative scores at the start of each part by repeat and broadcast operations
    for idx, sm in enumerate(score_matrices):
        sm = sm.repeat(num_demos ** idx, axis=1)
        if score_fn == 'sum':
            new_scores = sm + cumulative_scores[-1]
        elif score_fn == 'prod':
            new_scores = sm * cumulative_scores[-1]
        new_scores = new_scores.reshape(-1)
        cumulative_scores.append(new_scores)
    
    # Need to reverse this list so that scores are arranged in the direction of traversal
    # This will now be of the format [n ** 3, n**2, n] (3 part case) for n demonstrations
    cumulative_scores.reverse()
    
    return cumulative_scores

score_matrices_2parts = [score_matrix_2_part]
score_matrices_3parts = [score_matrix_3_part_1, score_matrix_3_part_2]

num_demos = score_matrix_2_part.shape[0]
scores_rear = np.ones((num_demos))

cumulative_scores_2_parts = {}
cumulative_scores_2_parts['sum'] = get_cumulative_scores(scores_rear, score_matrices_2parts, score_fn='sum')
cumulative_scores_2_parts['prod'] = get_cumulative_scores(scores_rear, score_matrices_2parts, score_fn='prod')

cumulative_scores_3_parts = {}
cumulative_scores_3_parts['sum'] = get_cumulative_scores(scores_rear, score_matrices_3parts, score_fn='sum')
cumulative_scores_3_parts['prod'] = get_cumulative_scores(scores_rear, score_matrices_3parts, score_fn='prod')

In [ ]:
print([len(x) for x in cumulative_scores_2_parts['sum']])
print([len(x) for x in cumulative_scores_3_parts['sum']])

## Compute best demonstration index using live score and cumulative_scores

1. 'score_front' is computed live
2. 'cm_score' contains the cumulative score (flattened) at the corresponding part index
3. 'factor' helps determine the correct index of the demonstration
4. 'score_front' is repeated 'factor' times to enable combining it with the cumulative score.
4. Then, an argmax operation helps find the correct index

In [ ]:
def get_best_part(score_front, cm_score, factor, step_value=None, score_fn='sum'):
    sf = score_front.repeat(factor)
    if score_fn == 'sum':
        temp = sf + cm_score
    elif score_fn == 'prod':
        temp = sf * cm_score
    
    return np.argmax(temp[0:factor * step_value]) // factor

## Evaluation Function 
We evaluate different combinations of experiments here:

Pick-n-Place task with sum and prod combinations for a different number of shape-sorting demonstrations

1. scores_front is pre-computed and stored (because this remains the same always)
2. New scores at next stages are computed on the go

In [ ]:
# Evaluation function
from math import pi
from flow_control.servoing.module import ServoingModule
from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.runner import evaluate_control
import ipdb
import cv2


selected_recordings = {}
for task in ['pick_n_place']:
    for score_fn in ['sum', 'prod']:
        for steps in [1, 5, 10, 15, 20]:
            selected_recordings[f'{task}_{score_fn}_{steps}'] = {}
        
def eval_cmb(playbacks, cumulative_scores, demo_good, live_seed, demo_parts, keypoint_info, step_val, score_fn, sf, num_parts=2):
    
    save_root = f'{data_dir}/multi_part_run/{num_parts}/{score_fn}/{step_val}'
    cscores = cumulative_scores[score_fn]
   
    # Instantiate env
    env = RobotSimEnv(task='shape_sorting', renderer=renderer, act_type='continuous',
                      initial_pose='close', max_steps=500, control='absolute-full',
                      img_size=(256, 256), param_randomize=("geom",), seed=int(live_seed),
                      task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]))
    
    # Mapping part to index
    traj_map = {0: 'locate', 1: 'insert'}
        
    save_dir = None
    
    # Total number of demonstrations that are used
    num_demos = scores_rear.shape[0]
    
    for idx in range(2):        
        state, _, _, _ = env.step(None)
        
        # Get the current gripper state
        current_rgb = state['rgb_gripper']
        
        if sf is None:
            if idx == 1:
                scores_front = compute_current_scores(playbacks, current_rgb, demo_parts, demo_good, traj_idx=idx, live_seed=live_seed)
#                 best_idx = 0
            else:
                print('Not possible to get here')
        else:
            
        # Compute live scores wrt the first image of the current part under consideration
#             scores_front = compute_current_scores(playbacks, current_rgb, demo_parts, demo_good, traj_idx=idx, live_seed=live_seed)

            # Compute a trajectory, get the demonstration (part) index that maximizes the score
            best_idx = get_best_part(sf, cscores[idx], num_demos ** (1 - idx), score_fn=score_fn, step_value=step_val)        
        best_demo = recordings[best_idx]
        
        # Keypoints for the demonstration part    
        kp_info = keypoint_info[best_idx]
        kps = kp_info[traj_map[idx]]
        
        if idx == 0:
            selected_recordings[f'{task}_{score_fn}_{step_val}'][live_seed] = [best_demo]
        
        if idx == 1:
            save_dir = f"{save_root}/run_pnp_{live_seed}_{best_idx}"
            folder_idx = 1
            updated_dir = save_dir
            while os.path.isdir(updated_dir):
                updated_dir = f"{save_dir}_{folder_idx}"
                folder_idx += 1
            
            save_dir = updated_dir
            selected_recordings[f'{task}_{score_fn}_{step_val}'][live_seed].append(best_demo)
            
        servo_module = ServoingModule(best_demo, control_config=control_config,
                                      start_paused=False, plot=False, plot_save_dir=None,
                                      load='select', selected_kp=kps)
        reward = 0
        _, reward, _, info = evaluate_control(env, servo_module, max_steps=130, save_dir=save_dir,
                                             initial_align=True if idx == 0 else False)
        
        sf = None
        
    del env
    del servo_module
    return reward

In [ ]:
def get_scores_front(seeds, demo_parts, demo_good):
    scores_front = []
    for idx, seed in enumerate(seeds):
        # Instantiate env
        env = RobotSimEnv(task='shape_sorting', renderer=renderer, act_type='continuous',
                          initial_pose='close', max_steps=500, control='absolute-full',
                          img_size=(256, 256), param_randomize=("geom",), seed=int(seed),
                          task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]))
        
        state, _, _, _ = env.step(None)
        
        # Get the current gripper state
        current_rgb = state['rgb_gripper']

        # Compute live scores wrt the first image of the current part under consideration
        sf = compute_current_scores(playbacks, current_rgb, demo_parts, demo_good, traj_idx=0, live_seed=seed)
        
        scores_front.append(sf)
        
        del env
        
    return scores_front

In [ ]:
# Get Servoing Rewards
num_live_seeds = 20

sel_recordings = {}
seeds = range(100, 100 + num_live_seeds)

scores_front_2parts = get_scores_front(seeds, demo_2parts, demo_good)

In [ ]:
np.save(f'{root_dir}/scores_front_2parts.npy', scores_front_2parts, allow_pickle=True)
# scores_front_2parts = np.load(f'{root_dir}/scores_front_2parts.npy', allow_pickle=True)

In [ ]:
all_rewards = {}  

for score_fn in ['sum', 'prod']:
    for step_idx, step_val in enumerate([1, 5, 10, 15, 20]):
        rewards = []
        for live_idx, live_seed in enumerate(range(num_live_seeds)):
            rew = eval_cmb(playbacks, cumulative_scores_2_parts, demo_good, live_seed, demo_2parts, keypoint_info_2parts, score_fn=score_fn, step_val=step_val, sf=scores_front_2parts[live_idx], num_parts=2)
            rewards.append(rew)
        all_rewards[f"{score_fn}_{step_val}"] = rewards

In [ ]:
object_uids = {'trapeze': 2, 'oval': 6}
def filter_run(pb, uid):
    return 1 if pb[-1].data['rew'] > 0 and np.atleast_1d(pb[-1].data['info'])[0]['object_selected'] == uid else 0
new_rewards = {}
for task in ['pick_n_place']:
    for score_fn in ['sum', 'prod']:
        for step_idx, step_val in enumerate([1, 5, 10, 15, 20]):
            run_dir = f'{data_dir}/multi_task_run_1vsN/{task}/{score_fn}/{step_val}'
            
            rewards = []
            for run in os.listdir(run_dir):            
                pb = PlaybackEnvServo(os.path.join(run_dir, run))
                rew = filter_run(pb, object_uids['trapeze'])
                rewards.append(rew)
            new_rewards[f"{task}_{score_fn}_{step_val}"] = rewards

In [ ]:
for key in new_rewards:
    print(key, np.mean(new_rewards[key]))

In [ ]:
import math
from matplotlib.ticker import MaxNLocator

steps = [1, 5, 10, 15, 20]
fig, ax = plt.subplots()
# ax = plt.figure().gca()
def format_fn(tick_val, tick_pos):
    if int(tick_val) in steps:
        return int(tick_val)
    else:
        return ''

# A FuncFormatter is created automatically.
# ax.xaxis.set_major_formatter(format_fn)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

for task in ["pick_n_place"]:
    mean_rew = {}
    for score_fn in ['sum', 'prod']:
        rew = []
        for step_idx, step_val in enumerate(steps):
            key = f"{task}_{score_fn}_{step_val}"
            rew.append(np.mean(new_rewards[key]))
        mean_rew[score_fn] = rew
        ax.plot(steps, rew, '.-', label=f'score_fn={score_fn}')
ax.set_ylim(ymin=0)

mean_rP, std_rP = 0.38, 0.19

ax.plot([0, 20], [mean_rP, mean_rP], "k--")
ax.axhspan(mean_rP - std_rP, mean_rP + std_rP, facecolor='gray', alpha=0.2)
ax.legend()
ax.set_title("Multi-Task Experiment: 1 'PNP' vs N 'SS'")
ax.set_xlabel("#Demonstrations")
ax.set_ylabel("Mean Rewards")
fig.savefig('multi_task.jpg', dpi=800)
# ax.show()